In [35]:
import praw
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

# Connecting to the Reddit API 

In [36]:
# Creating a reddit instance
reddit = praw.Reddit(client_id="EVgzl6l9hgixcX1QDiiN1g",
                     client_secret="wR1pXFdxhnpkBjockTDL3ykWGZsEDA",
                     user_agent="anything",
                     username="aryantiwari11", 
                     password="taryan1234589"
                    )


In [37]:
subreddit = reddit.subreddit("cryptomarkets")

In [38]:
# Printing the methods and attributes associated with the subreddit instance
print(dir(subreddit))

['MESSAGE_PREFIX', 'STR_FIELD', 'VALID_TIME_FILTERS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_convert_to_fancypants', '_create_or_update', '_fetch', '_fetch_data', '_fetch_info', '_fetched', '_kind', '_parse_xml_response', '_path', '_prepare', '_read_and_post_media', '_reddit', '_reset_attributes', '_safely_add_arguments', '_submission_class', '_submit_media', '_subreddit_collections_class', '_subreddit_list', '_upload_inline_media', '_upload_media', '_url_parts', '_validate_gallery', '_validate_inline_media', '_validate_time_filter', 'banned', 'collections', 'comments', 'contributor', 'controversial', 'display_name', 'emoji', 'filters', 'flair', 'fullname', 'gilded', 'h

In [39]:
hot_posts = subreddit.hot(limit=10)

In [40]:
print(dir(hot_posts))

['__abstractmethods__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_exhausted', '_extract_sublist', '_list_index', '_listing', '_next_batch', '_reddit', '_safely_add_arguments', 'limit', 'params', 'parse', 'url', 'yielded']


# Creating Sample Data
* Using only 10 hot posts to get an idea of the data being generated.
* After finalizing the features, data can be extracted from larger number of posts

In [41]:
sample_data = defaultdict(list)

for post in hot_posts:
    print(post.title)
    sample_data['title'].append(post.title)
    sample_data['score'].append(post.score)
    sample_data['url'].append(post.url)
    sample_data['num_comments'].append(post.num_comments)
    sample_data['text'].append(post.selftext)
    sample_data['created'].append(post.created)
    sample_data['flair'].append(post.link_flair_text)
    sample_data['original'].append(post.is_original_content)
    sample_data['subreddit'].append(post.subreddit)
    
    post.comments.replace_more(limit=None)
    all_comments = ""
    for comment in post.comments:
        all_comments += " " + comment.body
#         print("Added: ", comment.body)
    sample_data['comments'].append(all_comments)

Weekly Discussion Megathread - June 12, 2022 (GMT+0)
Celsius Withdrawals Stay Blocked for 5th Day as Clients Fear Losing All Funds
He must be sad (((
Elon Musk sued for $258 billion over dogecoin ‘pyramid scheme’
Celsius Network has now paused withdrawals, swaps, and transfers for over 72 hours now.
Circle to Launch Euro-pegged Stablecoin: Euro Coin
Less than a Month after Launching, Over 100 Token Pools Deployed on Bancor v3
Celsius can go bank themselves
Yahoo is Launching Metaverse Events in Hong Kong
MicroStrategy’s Michael Saylor On Bitcoin (BTC) Price: Now Is the “Ideal Entry Point” for Investors


In [43]:
len(sample_data)

10

In [44]:
import pandas as pd
sample_df = pd.DataFrame(sample_data)

In [45]:
sample_df.head()

,title,score,url,num_comments,text,created,flair,original,subreddit,comments
0,"Weekly Discussion Megathread - June 12, 2022 (...",5,https://www.reddit.com/r/CryptoMarkets/comment...,23,Welcome to the r/CryptoMarkets Weekly Discuss...,1.655039e+09,Discussion,False,CryptoMarkets,Today's Crypto Market status ：Extreme Fear! W...
1,Celsius Withdrawals Stay Blocked for 5th Day a...,91,https://tokenist.com/celsius-withdrawals-stay-...,16,,1.655390e+09,NEWS,False,CryptoMarkets,"In the FAQ email they just sent out, it says ..."
2,He must be sad (((,96,https://i.redd.it/08ibfyeafz591.jpg,13,,1.655385e+09,COMEDY,False,CryptoMarkets,He’s buying the dip Look at your own portfoli...
3,Elon Musk sued for $258 billion over dogecoin ...,15,https://nypost.com/2022/06/16/elon-musk-sued-f...,3,,1.655403e+09,None,False,CryptoMarkets,That should prove interesting considering the...
4,"Celsius Network has now paused withdrawals, sw...",13,https://www.reddit.com/r/CryptoMarkets/comment...,4,"Celsius Network has paused withdrawals, swaps,...",1.655393e+09,DISCUSSION,False,CryptoMarkets,"Without some takeovers or extreme limits, no...."


In [46]:
#Creating an additional feature of the polarity of the text using SIA
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
results = []

for line in sample_df['comments']:
    pol_score = sia.polarity_scores(line)
    pol_score['comments'] = line
    results.append(pol_score)

In [47]:
df2 = pd.DataFrame.from_records(results)

In [48]:
df2.head()

,neg,neu,pos,compound,comments
0,0.100,0.822,0.078,-0.6582,Today's Crypto Market status ：Extreme Fear! W...
1,0.173,0.687,0.140,-0.7482,"In the FAQ email they just sent out, it says ..."
2,0.116,0.792,0.093,-0.6597,He’s buying the dip Look at your own portfoli...
3,0.042,0.890,0.068,0.1027,That should prove interesting considering the...
4,0.088,0.912,0.000,-0.6265,"Without some takeovers or extreme limits, no...."


In [49]:
polarity = df2['compound']

In [50]:
sample_df = sample_df.join(polarity)
sample_df.rename(columns = {'compound':'polarity'}, inplace = True)

In [51]:
sample_df.head()

,title,score,url,num_comments,text,created,flair,original,subreddit,comments,polarity
0,"Weekly Discussion Megathread - June 12, 2022 (...",5,https://www.reddit.com/r/CryptoMarkets/comment...,23,Welcome to the r/CryptoMarkets Weekly Discuss...,1.655039e+09,Discussion,False,CryptoMarkets,Today's Crypto Market status ：Extreme Fear! W...,-0.6582
1,Celsius Withdrawals Stay Blocked for 5th Day a...,91,https://tokenist.com/celsius-withdrawals-stay-...,16,,1.655390e+09,NEWS,False,CryptoMarkets,"In the FAQ email they just sent out, it says ...",-0.7482
2,He must be sad (((,96,https://i.redd.it/08ibfyeafz591.jpg,13,,1.655385e+09,COMEDY,False,CryptoMarkets,He’s buying the dip Look at your own portfoli...,-0.6597
3,Elon Musk sued for $258 billion over dogecoin ...,15,https://nypost.com/2022/06/16/elon-musk-sued-f...,3,,1.655403e+09,None,False,CryptoMarkets,That should prove interesting considering the...,0.1027
4,"Celsius Network has now paused withdrawals, sw...",13,https://www.reddit.com/r/CryptoMarkets/comment...,4,"Celsius Network has paused withdrawals, swaps,...",1.655393e+09,DISCUSSION,False,CryptoMarkets,"Without some takeovers or extreme limits, no....",-0.6265


In [52]:
# Info about the length of the comments
total_posts = len(sample_df['comments'])
total_posts

inf = np.inf
max_len_comments = -inf
min_len_comments = inf
avg_len_comments = 0;

total_len_comments = 0
for comments in sample_df['comments']:
    total_len_comments += len(comments)
    max_len_comments = max(max_len_comments, len(comments))
    min_len_comments = min(min_len_comments, len(comments))

avg_len_comments = total_len_comments/total_posts

print("Max Length of comment: ", max_len_comments)
print("Min Length of comment: ", min_len_comments)
print("Average Length of comment: ", avg_len_comments)

Max Length of comment:  2992
Min Length of comment:  0
Average Length of comment:  673.5


# Data Preprocessing

In [53]:
df_copy = sample_df

In [54]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime as dt
from nltk.stem import PorterStemmer
import re
import string

In [55]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [56]:
df_copy['timestamp'] = df_copy['created'].apply(get_date)


In [57]:
def preprocess_text(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    
    text_tokens = word_tokenize(text)

    text_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            text_clean.append(stem_word)

    return text_clean

In [58]:
df_copy['title'] = df_copy['title'].apply(preprocess_text)
df_copy['comments'] = df_copy['comments'].apply(preprocess_text)
df_copy['text'] = df_copy['text'].apply(preprocess_text)


In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aryan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
df_copy.drop(['created'], axis=1, inplace=True)

In [61]:
df_copy.head()

,title,score,url,num_comments,text,flair,original,subreddit,comments,polarity,timestamp
0,"[weekli, discuss, megathread, june, 12, 2022, ...",5,https://www.reddit.com/r/CryptoMarkets/comment...,23,"[welcom, r/cryptomarket, weekli, discuss, thre...",Discussion,False,CryptoMarkets,"[today, 's, crypto, market, statu, ：extrem, fe...",-0.6582,2022-06-12 18:30:10
1,"[celsiu, withdraw, stay, block, 5th, day, clie...",91,https://tokenist.com/celsius-withdrawals-stay-...,16,[],NEWS,False,CryptoMarkets,"[in, faq, email, sent, say, investor, continu,...",-0.7482,2022-06-16 19:59:29
2,"[he, must, sad]",96,https://i.redd.it/08ibfyeafz591.jpg,13,[],COMEDY,False,CryptoMarkets,"[he, ’, buy, dip, look, portfolio, 😁, if, beli...",-0.6597,2022-06-16 18:36:18
3,"[elon, musk, su, billion, dogecoin, ‘, pyramid...",15,https://nypost.com/2022/06/16/elon-musk-sued-f...,3,[],None,False,CryptoMarkets,"[that, prove, interest, consid, take, suit, it...",0.1027,2022-06-16 23:31:44
4,"[celsiu, network, paus, withdraw, swap, transf...",13,https://www.reddit.com/r/CryptoMarkets/comment...,4,"[celsiu, network, paus, withdraw, swap, transf...",DISCUSSION,False,CryptoMarkets,"[without, takeov, extrem, limit, you, n't, bri...",-0.6265,2022-06-16 20:51:04


In [62]:
#Saving the data to a csv file
df_copy.to_csv('praw_sample_data.csv', index=False)

# Choosing the flairs
Picking only those flairs that appeared atleast 5 times in the subreddit

In [63]:
df = pd.read_csv('praw_sample_data.csv')
df.head(10)

,title,score,url,num_comments,text,flair,original,subreddit,comments,polarity,timestamp
0,"['weekli', 'discuss', 'megathread', 'june', '1...",5,https://www.reddit.com/r/CryptoMarkets/comment...,23,"['welcom', 'r/cryptomarket', 'weekli', 'discus...",Discussion,False,CryptoMarkets,"['today', ""'s"", 'crypto', 'market', 'statu', '...",-0.6582,2022-06-12 18:30:10
1,"['celsiu', 'withdraw', 'stay', 'block', '5th',...",91,https://tokenist.com/celsius-withdrawals-stay-...,16,[],NEWS,False,CryptoMarkets,"['in', 'faq', 'email', 'sent', 'say', 'investo...",-0.7482,2022-06-16 19:59:29
2,"['he', 'must', 'sad']",96,https://i.redd.it/08ibfyeafz591.jpg,13,[],COMEDY,False,CryptoMarkets,"['he', '’', 'buy', 'dip', 'look', 'portfolio',...",-0.6597,2022-06-16 18:36:18
3,"['elon', 'musk', 'su', 'billion', 'dogecoin', ...",15,https://nypost.com/2022/06/16/elon-musk-sued-f...,3,[],NaN,False,CryptoMarkets,"['that', 'prove', 'interest', 'consid', 'take'...",0.1027,2022-06-16 23:31:44
4,"['celsiu', 'network', 'paus', 'withdraw', 'swa...",13,https://www.reddit.com/r/CryptoMarkets/comment...,4,"['celsiu', 'network', 'paus', 'withdraw', 'swa...",DISCUSSION,False,CryptoMarkets,"['without', 'takeov', 'extrem', 'limit', 'you'...",-0.6265,2022-06-16 20:51:04
5,"['circl', 'launch', 'euro-peg', 'stablecoin', ...",12,https://coincodecap.com/circle-to-launch-euro-...,0,[],NaN,False,CryptoMarkets,[],0.0000,2022-06-16 22:01:47
6,"['less', 'month', 'launch', 'over', '100', 'to...",18,https://zycrypto.com/less-than-a-month-after-l...,1,[],NaN,False,CryptoMarkets,"['bancor', 'one', 'project', 'take', 'real', '...",-0.0258,2022-06-16 18:21:01
7,"['celsiu', 'go', 'bank']",7,https://youtube.com/shorts/zkCsNlOvDFk?feature...,2,[],NaN,False,CryptoMarkets,"['“', 'let', '’', 'put', 'alt', 'coin', 'aaaaa...",0.0000,2022-06-16 20:15:49
8,"['yahoo', 'launch', 'metavers', 'event', 'hong...",5,https://coincodecap.com/yahoo-is-launching-met...,0,[],NaN,False,CryptoMarkets,[],0.0000,2022-06-16 21:32:02
9,"['microstrategi', '’', 'michael', 'saylor', 'o...",136,https://timestabloid.com/microstrategys-michae...,62,[],NEWS,False,CryptoMarkets,"['bitcoin', 'pro', '/r/cryptomarkets/comments/...",0.9720,2022-06-16 02:12:07


In [64]:
df.describe()

,score,num_comments,polarity
count,10.000000,10.000000,10.000000
mean,39.800000,12.400000,-0.164370
std,48.435065,19.120669,0.527737
min,5.000000,0.000000,-0.748200
25%,8.250000,1.250000,-0.650275
50%,14.000000,3.500000,-0.012900
75%,72.750000,15.250000,0.000000
max,136.000000,62.000000,0.972000


In [65]:
df.isnull().sum()

title           0
score           0
url             0
num_comments    0
text            0
flair           5
original        0
subreddit       0
comments        0
polarity        0
timestamp       0
dtype: int64

In [66]:
flairs = []

for post in reddit.subreddit('cryptomarkets').top(time_filter="all", limit=500):
    flairs.append(post.link_flair_text)

In [67]:
df_flairs = pd.DataFrame(flairs)

In [68]:
df_flairs.value_counts()

COMEDY                        118
NEWS                          108
Comedy                         27
DISCUSSION                     26
News                           15
SENTIMENT                      14
FUNDAMENTALS                   10
Technical Analysis              9
TECHNICALS                      8
STRATEGY                        7
Educational                     6
Strategy                        5
Exchange                        5
Sentiment                       4
NEW COIN                        4
EXCHANGE                        3
Tool                            3
WARNING                         2
ANALYSIS                        2
Announcement                    2
Warning                         2
History                         1
Support-Open                    1
Fundmental Analysis             1
TOOL                            1
Fundamentals                    1
Discussion                      1
UNCONFIRMED                     1
Best Daily Meme - 09/23/19      1
dtype: int64

In [69]:
flair_count = {}

for i in flairs:
    if i in flair_count.keys():
        flair_count[i] += 1
    else:
        flair_count[i] = 1

In [74]:
final_flairs = []

for flair_unique in set(flairs):
    if(flair_count[flair_unique]>4):
        final_flairs.append(flair_unique)

final_flairs.remove(None)
        
print(final_flairs)

['News', 'FUNDAMENTALS', 'NEWS', 'DISCUSSION', 'TECHNICALS', 'Comedy', 'COMEDY', 'Exchange', 'SENTIMENT', 'STRATEGY', 'Strategy', 'Technical Analysis', 'Educational']


In [75]:
# Converting the flairs to a numeric format
flair_values = {}

for i in range(len(final_flairs)):
    flair_values[i] = final_flairs[i]

import pprint
pprint.pprint(flair_values)

{0: 'News',
 1: 'FUNDAMENTALS',
 2: 'NEWS',
 3: 'DISCUSSION',
 4: 'TECHNICALS',
 5: 'Comedy',
 6: 'COMEDY',
 7: 'Exchange',
 8: 'SENTIMENT',
 9: 'STRATEGY',
 10: 'Strategy',
 11: 'Technical Analysis',
 12: 'Educational'}


# Creating the final dataset

In [76]:
data_final = defaultdict(list)

count=1

for flair_key in flair_values:
    search_flair = subreddit.search(flair_values[flair_key], limit=100)
    for post in search_flair:
        data_final['flair'].append(flair_key)
        data_final['title'].append(post.title)
        data_final['url'].append(post.url)
        data_final['text'].append(post.selftext)
        data_final['score'].append(post.score)
        data_final['num_comments'].append(post.num_comments)
        data_final['original'].append(post.is_original_content)
        
        post.comments.replace_more(limit=None)
        all_comments = ''
        for comment in post.comments:
            all_comments = all_comments + ' ' + comment.body

        data_final['comments'].append(all_comments)
        
        print('Added ', count, 'samples')
        count+=1
        

Added  1 samples
Added  2 samples
Added  3 samples
Added  4 samples
Added  5 samples
Added  6 samples
Added  7 samples
Added  8 samples
Added  9 samples
Added  10 samples
Added  11 samples
Added  12 samples
Added  13 samples
Added  14 samples
Added  15 samples
Added  16 samples
Added  17 samples
Added  18 samples
Added  19 samples
Added  20 samples
Added  21 samples
Added  22 samples
Added  23 samples
Added  24 samples
Added  25 samples
Added  26 samples
Added  27 samples
Added  28 samples
Added  29 samples
Added  30 samples
Added  31 samples
Added  32 samples
Added  33 samples
Added  34 samples
Added  35 samples
Added  36 samples
Added  37 samples
Added  38 samples
Added  39 samples
Added  40 samples
Added  41 samples
Added  42 samples
Added  43 samples
Added  44 samples
Added  45 samples
Added  46 samples
Added  47 samples
Added  48 samples
Added  49 samples
Added  50 samples
Added  51 samples
Added  52 samples
Added  53 samples
Added  54 samples
Added  55 samples
Added  56 samples
A

Added  438 samples
Added  439 samples
Added  440 samples
Added  441 samples
Added  442 samples
Added  443 samples
Added  444 samples
Added  445 samples
Added  446 samples
Added  447 samples
Added  448 samples
Added  449 samples
Added  450 samples
Added  451 samples
Added  452 samples
Added  453 samples
Added  454 samples
Added  455 samples
Added  456 samples
Added  457 samples
Added  458 samples
Added  459 samples
Added  460 samples
Added  461 samples
Added  462 samples
Added  463 samples
Added  464 samples
Added  465 samples
Added  466 samples
Added  467 samples
Added  468 samples
Added  469 samples
Added  470 samples
Added  471 samples
Added  472 samples
Added  473 samples
Added  474 samples
Added  475 samples
Added  476 samples
Added  477 samples
Added  478 samples
Added  479 samples
Added  480 samples
Added  481 samples
Added  482 samples
Added  483 samples
Added  484 samples
Added  485 samples
Added  486 samples
Added  487 samples
Added  488 samples
Added  489 samples
Added  490 s

Added  870 samples
Added  871 samples
Added  872 samples
Added  873 samples
Added  874 samples
Added  875 samples
Added  876 samples
Added  877 samples
Added  878 samples
Added  879 samples
Added  880 samples
Added  881 samples
Added  882 samples
Added  883 samples
Added  884 samples
Added  885 samples
Added  886 samples
Added  887 samples
Added  888 samples
Added  889 samples
Added  890 samples
Added  891 samples
Added  892 samples
Added  893 samples
Added  894 samples
Added  895 samples
Added  896 samples
Added  897 samples
Added  898 samples
Added  899 samples
Added  900 samples
Added  901 samples
Added  902 samples
Added  903 samples
Added  904 samples
Added  905 samples
Added  906 samples
Added  907 samples
Added  908 samples
Added  909 samples
Added  910 samples
Added  911 samples
Added  912 samples
Added  913 samples
Added  914 samples
Added  915 samples
Added  916 samples
Added  917 samples
Added  918 samples
Added  919 samples
Added  920 samples
Added  921 samples
Added  922 s

In [80]:
df_final = pd.DataFrame(data_final)

In [87]:
df_final.head(2)

,flair,title,url,text,score,num_comments,original,comments
0,0,Ethereum Fees Now at July 2021 Lows of $2.54 -...,https://cryptonewsland.com/ethereum-fees-now-a...,,21,14,False,Ethereum [pros](/r/CryptoMarkets/comments/ux7...
1,0,Coinbase CEO’s Response to Employee Rant Backf...,https://cryptonewsland.com/coinbase-ceos-respo...,,4,7,False,USDC [pros](/r/CryptoMarkets/comments/v9izys/...


In [88]:
sia = SIA()
results = []

for line in df_final['comments']:
    pol_score = sia.polarity_scores(line)
    pol_score['comments'] = line
    results.append(pol_score)


In [89]:
df2 = pd.DataFrame.from_records(results)

In [90]:
df2.head()

,neg,neu,pos,compound,comments
0,0.038,0.785,0.177,0.9497,Ethereum [pros](/r/CryptoMarkets/comments/ux7...
1,0.168,0.759,0.073,-0.4404,USDC [pros](/r/CryptoMarkets/comments/v9izys/...
2,0.095,0.905,0.000,-0.2732,Bitcoin [pros](/r/CryptoMarkets/comments/v9vc...
3,0.031,0.877,0.092,0.6705,Do Kwan screwed us. They were just looking fo...
4,0.058,0.759,0.183,0.8979,Inflation [pros](/r/CryptoMarkets/comments/vd...


In [91]:
df_final = df_final.join(df2['compound'])

In [92]:
df_final.rename(columns = {'compound':'polarity'}, inplace = True)

In [93]:
df_final.head()

,flair,title,url,text,score,num_comments,original,comments,polarity
0,0,Ethereum Fees Now at July 2021 Lows of $2.54 -...,https://cryptonewsland.com/ethereum-fees-now-a...,,21,14,False,Ethereum [pros](/r/CryptoMarkets/comments/ux7...,0.9497
1,0,Coinbase CEO’s Response to Employee Rant Backf...,https://cryptonewsland.com/coinbase-ceos-respo...,,4,7,False,USDC [pros](/r/CryptoMarkets/comments/v9izys/...,-0.4404
2,0,Mars4 Metaverse is Selling Fast in Japan – Spo...,https://news.bitcoin.com/mars4-metaverse-is-se...,,2,9,False,Bitcoin [pros](/r/CryptoMarkets/comments/v9vc...,-0.2732
3,0,Terra Crisis: G-7 Nations Call for Urgent Cryp...,https://crypto.news/terra-g-7-nations-urgent-c...,,14,13,False,Do Kwan screwed us. They were just looking fo...,0.6705
4,0,White House says the Administration is Closely...,https://www.youtube.com/watch?v=4oSL1gCHFdQ,,0,4,False,Inflation [pros](/r/CryptoMarkets/comments/vd...,0.8979


In [94]:
df_final.to_csv('final_data2.csv', index=False)

In [95]:
len(df_final)

1120